In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/paperspace')
import torch

In [3]:
import fastai

In [4]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *

### Set PATH = directory where dogs and cats image data lives:

In [5]:
from fastai.conv_learner import *
PATH = '/home/paperspace/fastai/dogscats/'

### Image and batch sizes:

In [6]:
sz = 224; bs = 64

## Comparing the Resnet50 architecture to a Neural Net built using Keras

### Resnet50 comes first:

By default, all the layers except for the last few have frozen weights in the models downloaded from ImageNet.  This passes `test_name` to ImageClassifierData for future predictions.

This step takes a while. . .

In [7]:
tfms = tfms_from_model(resnet50, sz, aug_tfms = transforms_side_on, max_zoom = 1.1)
data = ImageClassifierData.from_paths(PATH, tfms = tfms, bs = bs, test_name = 'test1')
learn = ConvLearner.pretrained(resnet50, data)

# Three learning cycles with a learning rate of 1e-2.  No cycle length multiplication.
% time learn.fit(1e-2, 3, cycle_len = 1)

epoch      trn_loss   val_loss   accuracy                     
    0      0.042015   0.025537   0.991     
    1      0.038854   0.026472   0.991                        
    2      0.036191   0.023809   0.991                        

CPU times: user 16min 52s, sys: 18min 58s, total: 35min 50s
Wall time: 4min 46s


[array([0.02381]), 0.991]

#### Unfreeze layers from Resnet50 and a apply differential learning rates:

`bn_freeze` means that batch normalizations won't be updated. Don't need to do that if images largely conform to the Resnet architecture and are appropriate size (200 - 500px).

In [8]:
learn.unfreeze()
learn.bn_freeze(True) # sets the batch normalization to **not** be updated at each iteration. 
% time learn.fit([1e-5, 1e-4, 1e-2], 1, cycle_len = 1)

epoch      trn_loss   val_loss   accuracy                     
    0      0.018844   0.023606   0.991     

CPU times: user 7min 7s, sys: 6min 37s, total: 13min 45s
Wall time: 3min 32s


[array([0.02361]), 0.991]

#### Get predictions on validation set and score the model:

In [9]:
% time 
log_preds, y = learn.TTA() # running with test time augmentation.

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.5 µs


In [10]:
probs = np.exp(log_preds) # e^ on log_preds to translate them to probabilities

#### Resolve mismatch between log_preds formatting and y:

TTA does some image transformation when making the predictions, so it's changing the shape of the ouptut array that can be averaged together before scoring against y.

In [11]:
# look at y and log_preds shapes:
y.shape, probs.shape

((2000,), (5, 2000, 2))

In [12]:
# looks like log_preds contains a set of 5 prediction arrays from the test time augmentation above.
probs[0].shape

(2000, 2)

In [13]:
# average each set of predictions together over the first axis, which differentiates the prediction sets:
probs = np.mean(probs, axis=0)

In [14]:
probs.shape

(2000, 2)

In [15]:
probs[0:5] # array of probabilities for each class

array([[0.99997, 0.00003],
       [0.99955, 0.00045],
       [0.99998, 0.00002],
       [0.99951, 0.00049],
       [0.99998, 0.00002]], dtype=float32)

In [16]:
# convert probabilities to binary predictions on the target.  Second column represents class 1:
preds = probs[:, 1] > 0.50

In [17]:
metrics.log_loss(y, probs), np.mean(preds == y)

(0.016761740747447938, 0.994)

### Compare the above to an alternative built using Keras:

In [18]:
# install Tensorflow backend:
# the --prefix ensures that what's installed is usable from the current notebook.
#!conda install --yes --prefix {sys.prefix} tensorflow-gpu keras 

# Don't run like this: !conda install --yes numpy

# More info: https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /home/paperspace/anaconda3/envs/fastai:
#
keras                     2.1.5                    py36_0  
tensorflow-gpu            1.7.0                         0  


In [18]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D
from keras import backend as K

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
import numpy as np

In [21]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'
sz=224
batch_size = 64

#### Define data generators:
* augmentation
* normalization
* specify image size and mini-batch size for the image generator.  
* do the same thing for the validation_generator.  Be sure not to shuffle here because you need to track the prediction accuracy

In [22]:
train_datagen = ImageDataGenerator(rescale=1. / 255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)

In [23]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [24]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(sz, sz),
                                                    batch_size=batch_size, 
                                                    # could set class_mode = 'categorical for a multiclass problem
                                                    class_mode='binary')

Found 23000 images belonging to 2 classes.


In [25]:
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        shuffle=False,
                                                        target_size=(sz, sz),
                                                        batch_size=batch_size, 
                                                        class_mode='binary')

Found 2000 images belonging to 2 classes.


#### Make the base model:

In [26]:
# Keras doesn't include ResNet34, so use ResNet50 to compare the same models

base_model = ResNet50(weights = 'imagenet', include_top = False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(1, activation = 'sigmoid')(x)

#### Loop through and freeze the desired layers:

In [27]:
# compile the model:
model = Model(inputs = base_model.input, outputs = predictions)

for layer in base_model.layers: 
    layer.trainable = False

# Pass optimizer, loss function and evaluation metric:
model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

#### Fit the model:
* size per epoch
* number of workers
* batch size

In [ ]:
"""%%time
model.fit_generator(train_generator, 
                     train_generator.n / batch_size, 
                    epochs = 3, 
                    workers = 4,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.n / batch_size)
"""          

Epoch 1/3


#### Inspect model layers:

In [ ]:
#model.layers

#### Retrain some of the layers:

In [ ]:
#split_at = 140
#for layer in model.layers[:split_at]:
#    layer.trainable = False

#for layer in model.layers[split_at:]:
#    layer.trainable = True
    
#model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])